# Grover's Search Algorithm

Grover's algorithm is a quantum algorithm that finds a specific item in an *unsorted* list of *N* items with a quadratic speedup compared to classical search.  Instead of taking O(N) time (like a classical linear search), Grover's algorithm takes O(√N) time.  It achieves this by cleverly manipulating the amplitudes of the quantum states representing the items in the list.

**Key Concepts:**

*   **Oracle:**  A "black box" function that can recognize the item we're looking for.  In quantum terms, the oracle applies a negative phase to the state corresponding to the marked item.
*   **Diffusion Operator (Inversion About the Mean):**  An operation that amplifies the amplitude of the state marked by the oracle, while decreasing the amplitudes of the other states.
*   **Superposition:** The algorithm starts with a uniform superposition of all possible states (representing all items in the list).
*   **Iteration:** The oracle and diffusion operator are applied repeatedly (approximately √N times) to increase the probability of measuring the marked item.

**We will:**

1.  Implement the Grover's oracle.
2.  Implement the Grover diffusion operator.
3.  Combine these to build the complete Grover's search algorithm.
4.  Run the algorithm with an example.
5. Visualize results

## Step 1: Import Necessary Libraries

Before running the code, make sure you have the necessary libraries installed. You can install them using pip:


In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

We begin by importing the necessary libraries:

* **NumPy:**
   * `np`: For numerical operations (arrays, math functions).
*   **Qiskit:**
    *   `QuantumCircuit`: For building quantum circuits.
    *   `QuantumRegister`: For creating groups of qubits.
    *   `ClassicalRegister`: For storing measurement results.
    *   `transpile`: For circuit optimization.
*   **Qiskit Aer:**
     *    `AerSimulator`: For simulating quantum circuits.
*  **Qiskit Visualization**
   * `plot_histogram`: for vizualization
*   **Matplotlib:**
    *   `plt`: For plotting the results (histogram).

## Step 2: Define the Grover Oracle Function

The oracle marks the state corresponding to the item we're searching for.  It applies a negative phase (-1) to that state.

In [ ]:
def grover_oracle(n_qubits, marked_item):

    oracle = QuantumCircuit(n_qubits)
    # Convert the marked item to its binary representation (string)
    marked_item_binary = bin(marked_item)[2:].zfill(n_qubits)

    # Apply X gates to qubits where the marked item's binary rep is '0'
    for i, bit in enumerate(marked_item_binary):
        if bit == '0':
            oracle.x(i)

    # Apply a multi-controlled Z gate (MCZ)
    oracle.h(n_qubits - 1)
    oracle.mcx(list(range(n_qubits - 1)), n_qubits - 1)
    oracle.h(n_qubits - 1)

    # Apply X gates again to qubits where the binary rep is '0' (undo the first X)
    for i, bit in enumerate(marked_item_binary):
        if bit == '0':
            oracle.x(i)

    return oracle

The `grover_oracle` function constructs the oracle circuit:

*   **`def grover_oracle(n_qubits, marked_item):`**: Defines the function.
    *   `n_qubits`: The number of qubits.
    *   `marked_item`: The index of the item to be marked (an integer).

*   **`oracle = QuantumCircuit(n_qubits)`**: Creates a `QuantumCircuit` with `n_qubits`.

*   **`marked_item_binary = bin(marked_item)[2:].zfill(n_qubits)`**: Converts the `marked_item` (an integer) to its binary representation as a string.  For example, if `marked_item` is 5 and `n_qubits` is 3, `marked_item_binary` becomes "101". The `zfill(n_qubits)` part ensures the string has length `n_qubits` by padding with leading zeros if necessary.

*   **Apply X Gates (Conditional Negation):**
    *   `for i, bit in enumerate(marked_item_binary):`: Iterates through the bits of the binary representation.
    *   `if bit == '0': oracle.x(i)`: If a bit is '0', apply an X gate (NOT gate) to the corresponding qubit.  This step, and the identical one later, are key to making the oracle work correctly. We're essentially flipping the qubits to make the target state |11...1⟩, applying a phase flip to that state, and then flipping them back.

*   **Multi-Controlled Z (MCZ) Gate:** This is the core of the oracle. It applies a -1 phase to the |11...1⟩ state.
    *   `oracle.h(n_qubits - 1)`: Applies a Hadamard gate to the last qubit.
    *   `oracle.mcx(list(range(n_qubits - 1)), n_qubits - 1)`: Applies a multi-controlled X (Toffoli) gate.  This flips the last qubit *if and only if* all other qubits are in the |1⟩ state.
    *   `oracle.h(n_qubits - 1)`: Applies another Hadamard gate to the last qubit.  The combination of H, MCX, and H effectively implements a multi-controlled Z gate.

*   **Apply X Gates Again (Undo Conditional Negation):**  This undoes the X gates applied earlier, restoring the qubits to their original configuration *except* that the marked state now has a negative phase.

*   **`return oracle`**: Returns the constructed oracle circuit.

**How the Oracle Works (Key Idea):**
1.  We want to apply a negative phase *only* to the state that represents the `marked_item`.
2.  We use X gates to transform the `marked_item` state into the |11...1> state.
3.  We then apply a multi-controlled Z gate (implemented using H and MCX), to flip the sign.
4.  Finally, we use X gates to transform the state again, and return it to the original state.

## Step 3: Define the Grover Diffusion Operator Function

The diffusion operator (also called the inversion-about-the-mean operator) amplifies the amplitude of the marked state.

In [ ]:
def grover_diffusion(n_qubits):
    diffusion = QuantumCircuit(n_qubits)
    # Apply Hadamard to all qubits
    diffusion.h(range(n_qubits))

    # Apply X to all qubits
    diffusion.x(range(n_qubits))

    # Apply a multi-controlled Z gate (MCZ)
    diffusion.h(n_qubits - 1)
    diffusion.mcx(list(range(n_qubits - 1)), n_qubits - 1)  # Multi-controlled X (Toffoli)
    diffusion.h(n_qubits - 1)

    # Apply X to all qubits
    diffusion.x(range(n_qubits))

    # Apply Hadamard to all qubits
    diffusion.h(range(n_qubits))
    return diffusion

The `grover_diffusion` function constructs the diffusion operator:

*   **`def grover_diffusion(n_qubits):`**: Defines the function.
    *   `n_qubits`: The number of qubits.

*   **`diffusion = QuantumCircuit(n_qubits)`**: Creates a `QuantumCircuit`.

*   **Apply H to all qubits:** `diffusion.h(range(n_qubits))`

*   **Apply X to all qubits:** `diffusion.x(range(n_qubits))`

*   **Multi-Controlled Z (MCZ) Gate:**  Same as in the oracle, this applies a -1 phase to the |11...1⟩ state.  The combination of H, X, MCZ, X, and H implements the inversion-about-the-mean operation.
    *   `diffusion.h(n_qubits - 1)`
    *   `diffusion.mcx(list(range(n_qubits - 1)), n_qubits - 1)`
    *   `diffusion.h(n_qubits - 1)`

*   **Apply X to all qubits:** `diffusion.x(range(n_qubits))`

*   **Apply H to all qubits:** `diffusion.h(range(n_qubits))`

*   **`return diffusion`**: Returns the diffusion operator circuit.

**How the Diffusion Operator Works (Intuitively):**

1.  The initial Hadamards transform the state into a superposition where all states have equal amplitude (except the marked state, which has a negative amplitude due to the oracle).
2.  The X gates, MCZ gate, and X gates *reflect* the state vector about the *average* amplitude.  Since the marked state has a negative amplitude, this reflection amplifies its amplitude and reduces the amplitudes of the other states.
3.  The final Hadamards transform the state back to the computational basis.


## Step 4: Define the Grover Search Function

This function combines the oracle and diffusion operator to implement the full Grover's search algorithm.

In [ ]:
def grover_search(n_qubits, marked_item, num_iterations=None):
    if marked_item >= 2**n_qubits or marked_item < 0:
        raise ValueError("marked_item must be within the range [0, 2^n - 1]")

    # Initialize the quantum and classical registers
    qreg = QuantumRegister(n_qubits, 'q')
    creg = ClassicalRegister(n_qubits, 'c')
    circuit = QuantumCircuit(qreg, creg)

    # Apply Hadamard gates to all qubits to create a uniform superposition
    circuit.h(qreg)

    # Calculate the optimal number of iterations if not provided
    if num_iterations is None:
        num_iterations = int(np.round(np.pi / 4 * np.sqrt(2**n_qubits)))

    # Apply Grover iterations
    for _ in range(num_iterations):
        # Apply the oracle
        circuit.append(grover_oracle(n_qubits, marked_item), qreg)
        # Apply the diffusion operator
        circuit.append(grover_diffusion(n_qubits), qreg)

    # Measure all qubits
    circuit.measure(qreg, creg)


    # Simulate the circuit
    simulator = AerSimulator()
    compiled_circuit = transpile(circuit, simulator)
    job = simulator.run(compiled_circuit, shots=1024)
    result = job.result()
    counts = result.get_counts(circuit)

    return circuit, counts

The `grover_search` function implements the complete algorithm:

*   **`def grover_search(n_qubits, marked_item, num_iterations=None):`**: Defines the function.
    *   `n_qubits`: The number of qubits.
    *   `marked_item`: The index of the item to find.
    *   `num_iterations`: The number of times to apply the oracle and diffusion operator. If `None` (the default), the optimal number of iterations is calculated.

*   **Input Validation:**  Checks if `marked_item` is a valid index.

*   **Register Initialization:**
    *   `qreg = QuantumRegister(n_qubits, 'q')`: Creates a quantum register with `n_qubits`.
    *   `creg = ClassicalRegister(n_qubits, 'c')`: Creates a classical register with `n_qubits` bits (for measurement results).
    *   `circuit = QuantumCircuit(qreg, creg)`: Creates the main quantum circuit.

*   **Initial Superposition:** `circuit.h(qreg)`: Applies Hadamard gates to all qubits, creating a uniform superposition of all possible states.

*   **Optimal Iterations:**
    *   `if num_iterations is None:`: Checks if the number of iterations was provided.
    *   `num_iterations = int(np.round(np.pi / 4 * np.sqrt(2**n_qubits)))`: If not provided, calculates the *optimal* number of iterations.  This formula is derived from the analysis of Grover's algorithm.

*   **Grover Iterations:** `for _ in range(num_iterations):`:
    *   `circuit.append(grover_oracle(n_qubits, marked_item), qreg)`: Applies the oracle.
    *   `circuit.append(grover_diffusion(n_qubits), qreg)`: Applies the diffusion operator.

*   **Measurement:** `circuit.measure(qreg, creg)`: Measures all qubits and stores the results in the classical register.

* **Simulator and Counts**:
    *   `simulator = AerSimulator()`: Creates an `AerSimulator` object.
    *   `compiled_circuit = transpile(circuit, simulator)`: Optimizes the circuit for the simulator.
    *  `job = simulator.run(compiled_circuit, shots=1024)`: Runs the simulation with 1024 shots.
    *   `result = job.result()`: Gets the simulation results.
    * `counts = result.get_counts(circuit)`:  Retrieves a dictionary where keys are the measured bit strings (e.g., "0110") and values are the number of times each bit string was measured.

*   **Return Values:**
    *   `circuit`: The complete Grover's search circuit.
    *   `counts`: The measurement results (a dictionary of bit strings and their counts).


## Step 5: Main Function and Example Usage

In [ ]:
def main():
    # Example usage
    n_qubits = 3      # Size of the search space (2^3 = 8 elements)
    marked_item = 5   # The item we want to find (index 5, binary 101)

    # Run Grover's search
    circuit, counts = grover_search(n_qubits, marked_item)

    # Print the circuit (optional, for visualization)
    print(circuit.draw())

    # Print the measurement results
    print("\nMeasurement Results:")
    print(counts)

    # Plot the histogram of results
    # Use matplotlib.pyplot directly, handling potential ImportError
    try:
        plt.figure(figsize=(8, 6))  # Optional: Set figure size
        plt.bar(counts.keys(), counts.values())
        plt.xlabel("States")
        plt.ylabel("Counts")
        plt.title("Grover's Search Results")
        plt.xticks(rotation=90) #rotate labels
        plt.tight_layout()  # Adjust layout to prevent labels from overlapping
        plt.show()
    except ImportError:
        print("Matplotlib is not installed. Cannot plot histogram.")


    # Find the most frequent result (should be the marked item)
    most_frequent_result = max(counts, key=counts.get)
    print(f"\nMost frequent result (binary): {most_frequent_result}")
    print(f"Most frequent result (decimal): {int(most_frequent_result, 2)}")
    print(f"Target item: {marked_item}")

    if int(most_frequent_result, 2) == marked_item:
        print("Grover's search successful!")
    else:
        print("Grover's search did not find the target.")



if __name__ == "__main__":
    main()

*   **`def main():`**: Defines the main function where the example is executed.
    *   `n_qubits = 3`: Sets the number of qubits to 3 (search space size is 2³ = 8).
    *   `marked_item = 5`: Sets the item to search for to index 5 (binary representation is "101").

*   **Run Grover's Search:**
    *   `circuit, counts = grover_search(n_qubits, marked_item)`: Calls the `grover_search` function to create the circuit and get the measurement results.

*   **Print Circuit (Optional):**
    *   `print(circuit.draw())`: Prints a text-based representation of the quantum circuit.  This is useful for visualizing the circuit.

*   **Print Measurement Results:**
    *   `print("\nMeasurement Results:")`
    *   `print(counts)`: Prints the `counts` dictionary, showing how many times each state was measured.

* **Plot Histogram:**
 * The `try...except` block handles potential `ImportError` if `matplotlib` is not installed.
   * Inside the `try` block:
      * `plt.figure(figsize=(8,6))`: Sets the plot's figure size
      * `plt.bar(counts.keys(), counts.values())`: Creates a bar chart of results.
      * The rest of the code configures the plot labels and title.
    * If `matplotlib` is not found, the `except` block prints a message.
*   **Analyze Results:**
    *   `most_frequent_result = max(counts, key=counts.get)`: Finds the bit string that was measured most frequently.  This should be the marked item.
    *   Prints the most frequent result in both binary and decimal.
   *   Compares the most frequent result with the `marked_item` and prints whether the search was successful.

*   **`if __name__ == "__main__":`**:  This is a standard Python construct.  It ensures that the `main()` function is called only when the script is run directly (not when it's imported as a module).
